In [1]:
%load_ext autoreload
%autoreload 2

%cd '..'

/dss/dsshome1/03/di93fup/polarization_reddit


In [2]:
import random

from load.utils import load_comments
from load.constants import SEED

from sentence_transformers import SentenceTransformer
import numpy as np
from nltk.cluster import KMeansClusterer
from nltk.cluster.util import cosine_distance

In [3]:
model = SentenceTransformer('all-MiniLM-L6-v2')

In [4]:
YEAR = 2016
START_MONTH = 1
STOP_MONTH = 12

NUM_CLUSTERS = 7 # TBD: iterate over number of clusters

RNG = random.Random(SEED)

In [5]:
comments = load_comments(YEAR, start_month=START_MONTH, stop_month=STOP_MONTH)

Months: 100%|██████████| 1/1 [01:19<00:00, 79.29s/it]


In [6]:
comment_embeddings = model.encode(
    comments["body_cleaned"],
    show_progress_bar=True,
    normalize_embeddings=True,
)


Batches:   0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
comment_embeddings.shape

(50, 384)

In [8]:
print(NUM_CLUSTERS)

kclusterer = KMeansClusterer(
    num_means=NUM_CLUSTERS,
    distance=cosine_distance,
    rng=RNG,
)

assigned_clusters = kclusterer.cluster(comment_embeddings, assign_clusters=True)

means = np.array(kclusterer.means())

sum_dists = []
for i, c in enumerate(assigned_clusters):
    sum_dists.append(cosine_distance(means[c], comment_embeddings[i]))
print(np.mean(sum_dists))  # the smaller, the better


7
clustering...
calculating sum of distances...
0.47398783559183627
